In [8]:
import pandas as pd

inventory = '/home/onyxia/work/data/retail/retail_inventory_snapshot_30_10_25.csv' 
sales = '/home/onyxia/work/data/retail/retail_sales_data_01_09_2023_to_31_10_2025.csv'
output = 'output.csv'

# df_inventory = pd.read_csv(inventory, low_memory=False)
df_sales = pd.read_csv(sales, low_memory=False)

In [9]:
import pandas as pd
import io
import datetime
import numpy as np

def generate_daily_sales_report(csv_file_path, report_date):
    """
    Génère un rapport de ventes quotidien à partir d'un fichier CSV pour une date spécifique.

    Args:
        csv_file_path (str): Le chemin vers le fichier CSV.
        report_date (datetime.date): La date pour laquelle générer le rapport.
    """

    # 1. Lire les données CSV
    # Lit le fichier CSV directement à partir du chemin fourni
    try:
        df = pd.read_csv(csv_file_path, skipinitialspace=True)
    except FileNotFoundError:
        print(f"Erreur : Le fichier '{csv_file_path}' n'a pas été trouvé.")
        return
    except Exception as e:
        print(f"Erreur lors de la lecture du CSV : {e}")
        return

    # 2. Nettoyage et préparation des données
    try:
        # Convertir les colonnes pertinentes en types numériques, en gérant les erreurs
        numeric_cols = ['Turnover ex VAT', 'Disc Amount', 'Qty Sold', 'Profit']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # Remplir les valeurs non numériques (NaN) par 0
        df[numeric_cols] = df[numeric_cols].fillna(0)

        # Convertir la date de vente
        df['Sale Date'] = pd.to_datetime(df['Sale Date'], errors='coerce')

        # Nettoyer les noms des succursales (supprimer les espaces blancs)
        df['Branch Name'] = df['Branch Name'].str.strip()

        # Créer la colonne 'Gross Sales' (Ventes Brutes = Ventes Nettes + Réductions)
        df['Gross Sales'] = df['Turnover ex VAT'] + df['Disc Amount']

    except KeyError as e:
        print(f"Erreur : Colonne manquante dans le CSV - {e}")
        return
    except Exception as e:
        print(f"Erreur lors de la préparation des données : {e}")
        return

    # 3. Filtrer les données pour le jour du rapport
    daily_data = df[df['Sale Date'].dt.date == report_date].copy()

    if daily_data.empty:
        print(f"Aucune donnée trouvée pour le {report_date.strftime('%Y-%m-%d')}")
    
    # 4. Agréger les données par succursale
    # Note : 'nunique()' compte le nombre d'ID de vente uniques pour 'Trans Count'
    summary = daily_data.groupby('Branch Name').agg(
        Trans_Count=('Sale ID', 'nunique'),
        Total_Units=('Qty Sold', 'sum'),
        Gross_Sales=('Gross Sales', 'sum'),
        Discounts=('Disc Amount', 'sum'),
        Net_Sales=('Turnover ex VAT', 'sum'),
        Margin=('Profit', 'sum')
    )

    # 5. Calculer les métriques dérivées (Moyenne, Marge %)
    # Gérer la division par zéro en remplaçant inf/NaN par 0
    summary['Avg_Trans'] = (summary['Net_Sales'] / summary['Trans_Count']).replace([np.inf, -np.inf, np.nan], 0)
    summary['Margin_Percent'] = (summary['Margin'] / summary['Net_Sales'] * 100).replace([np.inf, -np.inf, np.nan], 0)

    # 6. Formater et imprimer le rapport
    
    # Définir la liste des emplacements comme dans le modèle
    locations = [
        "Baggot St", "Barrow St", "Bhagwans", "Castletymon",
        "Churchtown", "Glenview", "Kinvara", "Mater Hospital",
        "Ranelagh Village", "Sandford Rd"
    ]

    # Date et jour formatés
    report_date_str = report_date.strftime('%Y-%m-%d')
    business_day_str = report_date.strftime('%A, %B %d, %Y')

    # Imprimer l'en-tête du rapport
    print("DAILY SALES REPORT - MANUAL HUMAN REPORT")
    print(f"Report Date: {report_date_str}")
    print(f"Business Day: {business_day_str}")
    print("\n")

    # Imprimer l'en-tête du tableau
    header = "{:<20} | {:>12} | {:>12} | {:>15} | {:>12} | {:>15} | {:>12} | {:>12} | {:>10} | {:>7} | {:>10} | {:>12}".format(
        "Location", "Trans Count", "Total Units", "Gross Sales €", "Discounts €", "Net Sales €", 
        "Avg Trans €", "Margin €", "Margin %", "YoY %", "Target €", "vs Target %"
    )
    print(header)
    print("-" * len(header))

    # Initialiser les totaux
    total_trans = 0
    total_units = 0
    total_gross = 0.0
    total_discounts = 0.0
    total_net = 0.0
    total_margin = 0.0

    # Métriques non calculables à partir du CSV
    na_metrics = "{:>7} | {:>10} | {:>12}".format("N/A", "N/A", "N/A")

    # Parcourir chaque emplacement pour garantir le bon ordre
    for loc in locations:
        if loc in summary.index:
            # Obtenir les données pour cet emplacement
            data = summary.loc[loc]
            
            # Ajouter aux totaux
            total_trans += data['Trans_Count']
            total_units += data['Total_Units']
            total_gross += data['Gross_Sales']
            total_discounts += data['Discounts']
            total_net += data['Net_Sales']
            total_margin += data['Margin']

            # Formater la ligne
            row_str = "{:<20} | {:>12.0f} | {:>12.0f} | {:>15.2f} | {:>12.2f} | {:>15.2f} | {:>12.2f} | {:>12.2f} | {:>9.1f}% | {}".format(
                loc, data['Trans_Count'], data['Total_Units'], data['Gross_Sales'],
                data['Discounts'], data['Net_Sales'], data['Avg_Trans'], data['Margin'],
                data['Margin_Percent'], na_metrics
            )
            print(row_str)
        else:
            # Imprimer une ligne vide pour les emplacements sans ventes
            row_str = "{:<20} | {:>12} | {:>12} | {:>15} | {:>12} | {:>15} | {:>12} | {:>12} | {:>10} | {}".format(
                loc, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, "0.0%", na_metrics
            )
            print(row_str)

    # Calculer les moyennes totales
    total_avg_trans = (total_net / total_trans) if total_trans > 0 else 0.0
    total_margin_percent = (total_margin / total_net * 100) if total_net > 0 else 0.0

    # Imprimer la ligne TOTAL IN-STORE
    print("-" * len(header))
    total_row_str = "{:<20} | {:>12.0f} | {:>12.0f} | {:>15.2f} | {:>12.2f} | {:>15.2f} | {:>12.2f} | {:>12.2f} | {:>9.1f}% | {}".format(
        "TOTAL IN-STORE", total_trans, total_units, total_gross,
        total_discounts, total_net, total_avg_trans, total_margin,
        total_margin_percent, na_metrics
    )
    print(total_row_str)
    print("\n")

    # Imprimer les lignes Online et Grand Total (en supposant qu'elles sont les mêmes que le total pour l'instant)
    print("{:<20} | {:>12} | {:>12} | {:>15} | {:>12} | {:>15} | {:>12} | {:>12} | {:>10} | {}".format(
                "Online (Shopify)", 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, "0.0%", na_metrics
            ))
    print("=" * len(header))
    print("{:<20} | {:>12.0f} | {:>12.0f} | {:>15.2f} | {:>12.2f} | {:>15.2f} | {:>12.2f} | {:>12.2f} | {:>9.1f}% | {}".format(
        "GRAND TOTAL", total_trans, total_units, total_gross,
        total_discounts, total_net, total_avg_trans, total_margin,
        total_margin_percent, na_metrics
    ))

In [10]:
# --- EXÉCUTION DU SCRIPT ---

# 1. Définir le chemin vers votre fichier CSV
# REMPLACEZ CECI par le chemin réel de votre fichier
YOUR_CSV_FILE_PATH = sales

# Les données d'exemple (csv_data) ont été supprimées
# car nous lisons maintenant directement le fichier.

# 2. Définir la date pour laquelle exécuter le rapport
# Les données d'exemple sont du 2023-09-01
REPORT_DATE_TO_RUN = datetime.date(2023, 9, 1)

# 3. Appeler la fonction pour générer le rapport
generate_daily_sales_report(YOUR_CSV_FILE_PATH, REPORT_DATE_TO_RUN)

DAILY SALES REPORT - MANUAL HUMAN REPORT
Report Date: 2023-09-01
Business Day: Friday, September 01, 2023


Location             |  Trans Count |  Total Units |   Gross Sales € |  Discounts € |     Net Sales € |  Avg Trans € |     Margin € |   Margin % |   YoY % |   Target € |  vs Target %
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Baggot St            |          203 |          422 |         4963.74 |       410.36 |         4553.38 |        22.43 |      1606.88 |      35.3% |     N/A |        N/A |          N/A
Barrow St            |          168 |          271 |         2688.00 |       204.58 |         2483.42 |        14.78 |       935.56 |      37.7% |     N/A |        N/A |          N/A
Bhagwans             |            0 |            0 |             0.0 |          0.0 |             0.0 |          0.0 |          0.0 |       0.0% |     N/A |    

In [11]:
def generate_daily_sales_report(csv_file_path, report_date):
    """
    Génère un rapport de ventes quotidien à partir d'un fichier CSV pour une date spécifique.

    Args:
        csv_file_path (str): Le chemin vers le fichier CSV.
        report_date (datetime.date): La date pour laquelle générer le rapport.
    """

    # 1. Lire les données CSV
    # Lit le fichier CSV directement à partir du chemin fourni
    try:
        df = pd.read_csv(csv_file_path, skipinitialspace=True)
    except FileNotFoundError:
        print(f"Erreur : Le fichier '{csv_file_path}' n'a pas été trouvé.")
        return
    except Exception as e:
        print(f"Erreur lors de la lecture du CSV : {e}")
        return

    # 2. Nettoyage et préparation des données
    try:
        # Convertir les colonnes pertinentes en types numériques, en gérant les erreurs
        numeric_cols = ['Turnover ex VAT', 'Disc Amount', 'Qty Sold', 'Profit']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # Remplir les valeurs non numériques (NaN) par 0
        df[numeric_cols] = df[numeric_cols].fillna(0)

        # Convertir la date de vente
        df['Sale Date'] = pd.to_datetime(df['Sale Date'], errors='coerce')

        # Nettoyer les noms des succursales (supprimer les espaces blancs)
        df['Branch Name'] = df['Branch Name'].str.strip()

        # Créer la colonne 'Gross Sales' (Ventes Brutes = Ventes Nettes + Réductions)
        df['Gross Sales'] = df['Turnover ex VAT'] + df['Disc Amount']

    except KeyError as e:
        print(f"Erreur : Colonne manquante dans le CSV - {e}")
        return
    except Exception as e:
        print(f"Erreur lors de la préparation des données : {e}")
        return

    # 3. Filtrer les données pour le jour du rapport
    daily_data = df[df['Sale Date'].dt.date == report_date].copy()

    if daily_data.empty:
        print(f"Aucune donnée trouvée pour le {report_date.strftime('%Y-%m-%d')}")
    
    # 4. Agréger les données par succursale
    # Note : 'nunique()' compte le nombre d'ID de vente uniques pour 'Trans Count'
    summary = daily_data.groupby('Branch Name').agg(
        Trans_Count=('Sale ID', 'nunique'),
        Total_Units=('Qty Sold', 'sum'),
        Gross_Sales=('Gross Sales', 'sum'),
        Discounts=('Disc Amount', 'sum'),
        Net_Sales=('Turnover ex VAT', 'sum'),
        Margin=('Profit', 'sum')
    )

    # 5. Calculer les métriques dérivées (Moyenne, Marge %)
    # Gérer la division par zéro en remplaçant inf/NaN par 0
    summary['Avg_Trans'] = (summary['Net_Sales'] / summary['Trans_Count']).replace([np.inf, -np.inf, np.nan], 0)
    summary['Margin_Percent'] = (summary['Margin'] / summary['Net_Sales'] * 100).replace([np.inf, -np.inf, np.nan], 0)

    # 6. Formater et imprimer le rapport
    
    # Définir la liste des emplacements comme dans le modèle
    locations = [
        "Baggot St", "Barrow St", "Bhagwans", "Castletymon",
        "Churchtown", "Glenview", "Kinvara", "Mater Hospital",
        "Ranelagh Village", "Sandford Rd"
    ]

    # Date et jour formatés
    report_date_str = report_date.strftime('%Y-%m-%d')
    business_day_str = report_date.strftime('%A, %B %d, %Y')

    # Imprimer l'en-tête du rapport
    print("DAILY SALES REPORT - MANUAL HUMAN REPORT")
    print(f"Report Date: {report_date_str}")
    print(f"Business Day: {business_day_str}")
    print("\n")

    # Imprimer l'en-tête du tableau
    header = "{:<20} | {:>12} | {:>12} | {:>15} | {:>12} | {:>15} | {:>12} | {:>12} | {:>10} | {:>7} | {:>10} | {:>12}".format(
        "Location", "Trans Count", "Total Units", "Gross Sales €", "Discounts €", "Net Sales €", 
        "Avg Trans €", "Margin €", "Margin %", "YoY %", "Target €", "vs Target %"
    )
    print(header)
    print("-" * len(header))

    # Initialiser les totaux
    total_trans = 0
    total_units = 0
    total_gross = 0.0
    total_discounts = 0.0
    total_net = 0.0
    total_margin = 0.0

    # Métriques non calculables à partir du CSV
    na_metrics = "{:>7} | {:>10} | {:>12}".format("N/A", "N/A", "N/A")

    # Parcourir chaque emplacement pour garantir le bon ordre
    for loc in locations:
        if loc in summary.index:
            # Obtenir les données pour cet emplacement
            data = summary.loc[loc]
            
            # Ajouter aux totaux
            total_trans += data['Trans_Count']
            total_units += data['Total_Units']
            total_gross += data['Gross_Sales']
            total_discounts += data['Discounts']
            total_net += data['Net_Sales']
            total_margin += data['Margin']

            # Formater la ligne
            row_str = "{:<20} | {:>12.0f} | {:>12.0f} | {:>15.2f} | {:>12.2f} | {:>15.2f} | {:>12.2f} | {:>12.2f} | {:>9.1f}% | {}".format(
                loc, data['Trans_Count'], data['Total_Units'], data['Gross_Sales'],
                data['Discounts'], data['Net_Sales'], data['Avg_Trans'], data['Margin'],
                data['Margin_Percent'], na_metrics
            )
            print(row_str)
        else:
            # Imprimer une ligne vide pour les emplacements sans ventes
            row_str = "{:<20} | {:>12} | {:>12} | {:>15} | {:>12} | {:>15} | {:>12} | {:>12} | {:>10} | {}".format(
                loc, 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, "0.0%", na_metrics
            )
            print(row_str)

    # Calculer les moyennes totales
    total_avg_trans = (total_net / total_trans) if total_trans > 0 else 0.0
    total_margin_percent = (total_margin / total_net * 100) if total_net > 0 else 0.0

    # Imprimer la ligne TOTAL IN-STORE
    print("-" * len(header))
    total_row_str = "{:<20} | {:>12.0f} | {:>12.0f} | {:>15.2f} | {:>12.2f} | {:>15.2f} | {:>12.2f} | {:>12.2f} | {:>9.1f}% | {}".format(
        "TOTAL IN-STORE", total_trans, total_units, total_gross,
        total_discounts, total_net, total_avg_trans, total_margin,
        total_margin_percent, na_metrics
    )
    print(total_row_str)
    print("\n")

    # Imprimer les lignes Online et Grand Total (en supposant qu'elles sont les mêmes que le total pour l'instant)
    print("{:<20} | {:>12} | {:>12} | {:>15} | {:>12} | {:>15} | {:>12} | {:>12} | {:>10} | {}".format(
                "Online (Shopify)", 0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, "0.0%", na_metrics
            ))
    print("=" * len(header))
    print("{:<20} | {:>12.0f} | {:>12.0f} | {:>15.2f} | {:>12.2f} | {:>15.2f} | {:>12.2f} | {:>12.2f} | {:>9.1f}% | {}".format(
        "GRAND TOTAL", total_trans, total_units, total_gross,
        total_discounts, total_net, total_avg_trans, total_margin,
        total_margin_percent, na_metrics
    ))


def generate_product_performance_report(csv_file_path, start_date, end_date, top_n=50):
    """
    Génère un rapport de performance produit pour une plage de dates,
    trié par ventes nettes.

    Args:
        csv_file_path (str): Le chemin vers le fichier CSV.
        start_date (datetime.date): La date de début de la période.
        end_date (datetime.date): La date de fin de la période.
        top_n (int): Le nombre de produits à classer (par ex. 50).
    """

    # 1. Lire les données CSV
    try:
        # Ajout de 'low_memory=False' pour éviter les avertissements sur les types mixtes
        df = pd.read_csv(csv_file_path, skipinitialspace=True, low_memory=False)
    except FileNotFoundError:
        print(f"Erreur : Le fichier '{csv_file_path}' n'a pas été trouvé.")
        return
    except Exception as e:
        print(f"Erreur lors de la lecture du CSV : {e}")
        return

    # 2. Nettoyage et préparation des données
    try:
        # Colonnes pour les calculs
        # 'Trade Price' est ajouté pour le coût, au cas où Profit = 0
        numeric_cols = ['Turnover ex VAT', 'Disc Amount', 'Qty Sold', 'Profit', 'Trade Price']
        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        df[numeric_cols] = df[numeric_cols].fillna(0)

        # Colonnes pour le regroupement (nettoyage des espaces)
        group_cols = ['Headoffice ID', 'Product', 'Dept Fullname', 'OrderList']
        for col in group_cols:
            df[col] = df[col].astype(str).str.strip()

        # Convertir la date de vente
        df['Sale Date'] = pd.to_datetime(df['Sale Date'], errors='coerce')
        
        # Supprimer les lignes où la date n'a pas pu être analysée
        df = df.dropna(subset=['Sale Date'])

        # Créer les colonnes calculées nécessaires
        df['Gross Sales'] = df['Turnover ex VAT'] + df['Disc Amount']
        
        # Calcul du coût : Ventes Nettes - Marge.
        # C'est la méthode la plus précise si la Marge est correcte.
        df['Total Cost'] = (df['Turnover ex VAT'] - df['Profit'])
        
        # Cas de secours : Si le coût est nul (par ex. Marge = Ventes Nettes), 
        # utilisons 'Trade Price' * 'Qty Sold' comme approximation
        df.loc[df['Total Cost'] == 0, 'Total Cost'] = df['Trade Price'] * df['Qty Sold']


    except KeyError as e:
        print(f"Erreur : Colonne manquante dans le CSV - {e}")
        return
    except Exception as e:
        print(f"Erreur lors de la préparation des données : {e}")
        return

    # 3. Filtrer les données pour la plage de dates
    period_data = df[(df['Sale Date'].dt.date >= start_date) & 
                     (df['Sale Date'].dt.date <= end_date)].copy()

    if period_data.empty:
        print(f"Aucune donnée trouvée pour la période du {start_date.strftime('%Y-%m-%d')} au {end_date.strftime('%Y-%m-%d')}")
        # On continue pour imprimer un rapport vide
    
    # 4. Agréger les données par produit
    # Nous regroupons par les identifiants de produit
    product_summary = period_data.groupby([
        'Headoffice ID',  # SKU
        'Product',        # Product Name
        'Dept Fullname',  # Department
        'OrderList'       # Supplier
    ]).agg(
        Units_Sold=('Qty Sold', 'sum'),
        Gross_Sales=('Gross Sales', 'sum'),
        Discounts=('Disc Amount', 'sum'),
        Net_Sales=('Turnover ex VAT', 'sum'),
        Cost=('Total Cost', 'sum'),
        Margin=('Profit', 'sum')
    ).reset_index()

    # 5. Calculer les métriques dérivées
    product_summary['Margin_Percent'] = (product_summary['Margin'] / product_summary['Net_Sales'] * 100).replace([np.inf, -np.inf, np.nan], 0)
    product_summary['Avg_Price'] = (product_summary['Net_Sales'] / product_summary['Units_Sold']).replace([np.inf, -np.inf, np.nan], 0)
    
    # 6. Trier par Ventes Nettes pour obtenir le classement
    product_summary = product_summary.sort_values(by='Net_Sales', ascending=False)
    
    # Ajouter le classement
    product_summary['Rank'] = np.arange(1, len(product_summary) + 1)
    
    # 7. Préparer les données finales du rapport (Top N)
    report_data = product_summary.head(top_n)

    # 8. Calculer les totaux (basés sur *tous* les produits, pas seulement le Top N)
    total_units = product_summary['Units_Sold'].sum()
    total_gross = product_summary['Gross_Sales'].sum()
    total_discounts = product_summary['Discounts'].sum()
    total_net = product_summary['Net_Sales'].sum()
    total_cost = product_summary['Cost'].sum()
    total_margin = product_summary['Margin'].sum()
    
    total_margin_percent = (total_margin / total_net * 100) if total_net > 0 else 0.0
    total_avg_price = (total_net / total_units) if total_units > 0 else 0.0

    # 9. Formater et imprimer le rapport
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    print("PRODUCT PERFORMANCE ANALYSIS - MANUAL HUMAN REPORT")
    print(f"Period: {start_date_str} to {end_date_str}")
    print("\n")

    # Définir le format de l'en-tête (très large)
    header_format = (
        "{:<6} | {:<10} | {:<40} | {:<20} | {:<25} | "
        "{:>10} | {:>15} | {:>12} | {:>15} | {:>12} | {:>12} | {:>10} | {:>12} | {:>15}"
    )
    header = header_format.format(
        "Rank", "SKU", "Product Name", "Department", "Supplier", 
        "Units Sold", "Gross Sales €", "Discounts €", "Net Sales €", 
        "Cost €", "Margin €", "Margin %", "Avg Price €", "Stock on Hand"
    )
    print(header)
    print("-" * len(header))

    # Métrique non calculable
    na_metric_stock = "{:>15}".format("N/A")

    # Format pour les lignes de données
    data_format = (
        "{:<6} | {:<10} | {:<40.40} | {:<20.20} | {:<25.25} | "
        "{:>10.0f} | {:>15.2f} | {:>12.2f} | {:>15.2f} | {:>12.2f} | {:>12.2f} | {:>9.1f}% | {:>12.2f} | {}"
    )
    
    # Format pour les lignes vides
    empty_row_format = (
        "{:<6} | {:<10} | {:<40} | {:<20} | {:<25} | "
        "{:>10} | {:>15} | {:>12} | {:>15} | {:>12} | {:>12} | {:>10} | {:>12} | {:>15}"
    )
    
    # Créer un dictionnaire à partir de report_data pour un accès facile par rang
    report_dict = {row['Rank']: row for index, row in report_data.iterrows()}

    for i in range(1, top_n + 1):
        if i in report_dict:
            data = report_dict[i]
            row_str = data_format.format(
                data['Rank'],
                str(data['Headoffice ID']), # SKU
                str(data['Product']),       # Product Name
                str(data['Dept Fullname']), # Department
                str(data['OrderList']),     # Supplier
                data['Units_Sold'],
                data['Gross_Sales'],
                data['Discounts'],
                data['Net_Sales'],
                data['Cost'],
                data['Margin'],
                data['Margin_Percent'],
                data['Avg_Price'],
                na_metric_stock
            )
            print(row_str)
        else:
            # Imprimer une ligne vide pour les rangs sans données, correspondant au modèle
            print(empty_row_format.format(
                i, "", "", "", "", 0, 0.0, 0.0, 0.0, 0.0, 0.0, "0.0%", 0.0, "N/A"
            ))

    # Imprimer la ligne TOTAL
    print("-" * len(header))
    total_format = (
        "{:<6} | {:<10} | {:<40} | {:<20} | {:<25} | "
        "{:>10.0f} | {:>15.2f} | {:>12.2f} | {:>15.2f} | {:>12.2f} | {:>12.2f} | {:>9.1f}% | {:>12.2f} | {}"
    )
    total_str = total_format.format(
        "TOTAL", "", "", "", "",
        total_units,
        total_gross,
        total_discounts,
        total_net,
        total_cost,
        total_margin,
        total_margin_percent,
        total_avg_price,
        na_metric_stock
    )
    print(total_str)
    print("\n\n")
    
    # Imprimer le guide "HOW TO USE"
    print("📋 HOW TO USE THIS REPORT - COMPREHENSIVE GUIDE - WHAT HUMANS HAVE TO DO MANUALLY TODAY.")
    print("\nPURPOSE: The Product Performance Report is your essential merchandising...")
    # ... (Le reste du guide serait imprimé ici) ...
    # (J'omets le texte complet pour la brièveté de cette réponse, 
    # mais vous pouvez ajouter des instructions print() pour le reste du texte)
    print("\nREMEMBER: This report is the foundation of merchandising excellence...")

In [12]:
# --- EXÉCUTION DU SCRIPT ---

# 1. Définir le chemin vers votre fichier CSV
# REMPLACEZ CECI par le chemin réel de votre fichier
YOUR_CSV_FILE_PATH = sales

# Les données d'exemple (csv_data) ont été supprimées
# car nous lisons maintenant directement le fichier.

# 2. Définir la date pour laquelle exécuter le rapport quotidien
# Les données d'exemple sont du 2023-09-01
REPORT_DATE_TO_RUN = datetime.date(2023, 9, 1)

# 3. Appeler la fonction pour générer le rapport
print("--- DÉBUT DU RAPPORT DE VENTES QUOTIDIEN ---")
generate_daily_sales_report(YOUR_CSV_FILE_PATH, REPORT_DATE_TO_RUN)
print("--- FIN DU RAPPORT DE VENTES QUOTIDIEN ---\n\n")




--- DÉBUT DU RAPPORT DE VENTES QUOTIDIEN ---
DAILY SALES REPORT - MANUAL HUMAN REPORT
Report Date: 2023-09-01
Business Day: Friday, September 01, 2023


Location             |  Trans Count |  Total Units |   Gross Sales € |  Discounts € |     Net Sales € |  Avg Trans € |     Margin € |   Margin % |   YoY % |   Target € |  vs Target %
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Baggot St            |          203 |          422 |         4963.74 |       410.36 |         4553.38 |        22.43 |      1606.88 |      35.3% |     N/A |        N/A |          N/A
Barrow St            |          168 |          271 |         2688.00 |       204.58 |         2483.42 |        14.78 |       935.56 |      37.7% |     N/A |        N/A |          N/A
Bhagwans             |            0 |            0 |             0.0 |          0.0 |             0.0 |          0.

In [13]:
# 4. Nouvelle exécution pour le rapport de performance produit
print("--- DÉBUT DU RAPPORT DE PERFORMANCE PRODUIT ---")
# Définir la plage de dates
# Pour cet exemple, nous utilisons juste le même jour que ci-dessus
# Dans un cas réel, ce serait (par ex.) 30 jours avant
START_DATE = datetime.date(2023, 9, 1)
END_DATE = datetime.date(2023, 9, 1) 
generate_product_performance_report(YOUR_CSV_FILE_PATH, START_DATE, END_DATE, top_n=50)
print("--- FIN DU RAPPORT DE PERFORMANCE PRODUIT ---")

--- DÉBUT DU RAPPORT DE PERFORMANCE PRODUIT ---
PRODUCT PERFORMANCE ANALYSIS - MANUAL HUMAN REPORT
Period: 2023-09-01 to 2023-09-01


Rank   | SKU        | Product Name                             | Department           | Supplier                  | Units Sold |   Gross Sales € |  Discounts € |     Net Sales € |       Cost € |     Margin € |   Margin % |  Avg Price € |   Stock on Hand
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1      | 50421      | Symprove Mango & Passionfruit 4 Week Pac | Symprove             | Symprove UK               |          6 |          532.52 |        33.00 |          499.52 |       317.15 |       182.37 |      36.5% |        83.25 |             N/A
2      | 2371       | Solpadeine Soluble 24s                   | OTC : Analgesics     | Pharmax         